In [1200]:
! rm -f dale_chall.py
! wget https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py

--2021-11-21 20:02:37--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27456 (27K) [text/plain]
Saving to: ‘dale_chall.py’

dale_chall.py       100%[===================>]  26.81K  --.-KB/s    in 0.001s  

2021-11-21 20:02:37 (18.0 MB/s) - ‘dale_chall.py’ saved [27456/27456]



In [1201]:
!pip install pyphen nltk pandas sklearn openpyxl

In [1202]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from sklearn import preprocessing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1203]:
! rm -rf data*
! wget https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
! unzip "data.zip"

! echo "***\n Fisierele sunt: "
! ls data/
! echo "****\n Calea catre directorul cu date este: "
! readlink -f data/

--2021-11-21 20:02:40--  https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip [following]
--2021-11-21 20:02:40--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 741506 (724K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 724.13K  --.-KB/s    in 0.06s   

2021-11-21 20:02:41 (12.0 MB/s) - ‘data.zip’ saved [741506/741506]

Arc

In [1204]:
import pyphen
import numpy as np
import pandas as pd

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# puteti incerca si KNN de la laboratorul 3
from sklearn.neighbors import KNeighborsClassifier
# https://en.wikipedia.org/wiki/Precision_and_recall#Imbalanced_data
from sklearn.metrics import balanced_accuracy_score

# https://readabilityformulas.com/new-dale-chall-readability-formula.php
from dale_chall import DALE_CHALL

In [1205]:
dtypes = {"sentence": "string", "token": "string", "complexity": "float64"}
train = pd.read_excel('/content/data/train.xlsx', dtype=dtypes, keep_default_na=False)
test = pd.read_excel('/content/data/test.xlsx', dtype=dtypes, keep_default_na=False)

# **Word structure features**

In [1206]:
def is_dale_chall(word):
  '''Daca e in lista Dale-Chall sau nu.
  Atentie daca cuvantul e scris cu litera mare sau mica.
  '''
  return int(word in DALE_CHALL)
  
  
print(is_dale_chall('car'))
print(is_dale_chall('Car'))
print(is_dale_chall('supercalifragilistic'))

1
0
0


In [1207]:
def is_title(word):
  '''Indica ca avem de a face cu o entitate.
  Care de obicei au o complexitate conceptuala mai mare.
  '''
  return int(word.istitle())

print(is_title('Hello World'))
print(is_title('hoi'))

1
0


In [1208]:
import pyphen
dic = pyphen.Pyphen(lang='en')
def nr_syllables(word):
  return len(dic.inserted(word).split('-'))
  

In [1209]:
vowels = 'aeiouw'
def nr_vowels(word):
    nr = 0
    for chr in word:
        if chr in vowels:
            nr += 1
    return nr

In [1210]:
def get_word_structure_features(word):
    '''Caracteristici la nivelul structurii unui cuvant.
    '''
    features = []
    features.append(is_dale_chall(word))
    features.append(is_title(word))
    features.append(nr_syllables(word))
    features.append(len(word))
    features.append(nr_vowels(word))
    return np.array(features)

# **Corpus**

In [1211]:
def corpus_feature(corpus):
  '''E din corpusul bible, biomed sau europarl.
  Atentie sunt categorical features.
  '''
  #d = {"bible": [1, 0, 0], "europarl": [0, 1, 0], "biomed": [0, 0, 1]}
  d = {"bible": [0], "europarl": [1], "biomed": [2]}
  return d[corpus]

# **Wordnet**

In [1212]:
from nltk.corpus import wordnet as wn
def nr_synsets(word):
    return len(wn.synsets(word))
  

In [1213]:
def get_hyponyms(word):
  hyper= wn.synsets(word)
  sum=0
  for i in hyper:
    sum=sum+len(i.hypernyms())
  return sum

In [1214]:
def get_hypernyms(word):
  hypo= wn.synsets(word)
  sum=0
  for i in hypo:
    sum=sum+len(i.hyponyms())
  return sum

In [1215]:
def get_meronyms_parts(word):
  hypo= wn.synsets(word)
  sum=0
  for i in hypo:
    sum=sum+len(i.part_meronyms())
  return sum

In [1216]:
def get_meronyms_substances(word):
  hypo= wn.synsets(word)
  sum=0
  for i in hypo:
    sum=sum+len(i.substance_meronyms())
  return sum

In [1217]:
def get_holonyms_parts(word):
  holo= wn.synsets(word)
  sum=0
  for i in holo:
    sum=sum+len(i.part_holonyms())
  return sum

In [1218]:
def get_holonyms_substances(word):
  holo= wn.synsets(word)
  sum=0
  for i in holo:
    sum=sum+len(i.substance_holonyms())
  return sum

In [1219]:
def get_wordnet_features(word):
  features = []
  features.append(nr_synsets(word))
  features.append(get_hyponyms(word))
  features.append(get_hypernyms(word))
  features.append(get_meronyms_parts(word))
  features.append(get_meronyms_substances(word))
  features.append(get_holonyms_parts(word))
  features.append(get_holonyms_substances(word))
  return np.array(features)

In [1220]:
# test = wn.synsets('bird')
# print(test)
# sum=0
# for i in test:
#   sum=sum+len(i.hyponyms())

# print(sum)





# **Context**

In [1221]:
def nearby_words_complexity(sentence):
  word = sentence.split(' ')
  features = []
  for i in word:
    features.append(get_word_structure_features(i)[0])
  features= np.mean(features)
  return features

print(nearby_words_complexity("Azi merg la mare"))


0.25


In [1222]:
def get_context_features(sentence):
  features = []
  features.append(nearby_words_complexity(sentence))
  return np.array(features)



# **Append all Features**

In [1223]:
def featurize(row):
    word = row['token']
    all_features = []
    all_features.extend(corpus_feature(row['corpus']))
    all_features.extend(get_word_structure_features(word))
    all_features.extend(get_wordnet_features(word))
    all_features.extend(get_context_features(row['sentence']))
    return np.array(all_features)

In [1224]:
def featurize_df(df):
    nr_of_features = len(featurize(df.iloc[0]))
    print('nr de features este ', nr_of_features)
    nr_of_examples = len(df)
    print('nr de exemple este ', nr_of_examples)
    features = np.zeros((nr_of_examples, nr_of_features))
    for index, row in df.iterrows():
        row_ftrs = featurize(row)
        features[index, :] = row_ftrs
    return features

In [1225]:
from sklearn import preprocessing
X_train = preprocessing.normalize(featurize_df(train))
print('Datele de antrenare au forma ', X_train.shape)
y_train = train['complex'].values
print('Etichetele datelor de antrenare arata cam asa: ', y_train)

nr de features este  14
nr de exemple este  7662
Datele de antrenare au forma  (7662, 14)
Etichetele datelor de antrenare arata cam asa:  [0 0 0 ... 1 1 1]


# **TESTARE**

In [1226]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.model_selection import train_test_split
X_traincopy=X_train
y_traincopy= y_train
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train2, y_train2)

preds = model.predict(X_valid)

print(balanced_accuracy_score(y_valid, preds))

0.6730529230529231


# **APLICARE**

In [1227]:
X_test = preprocessing.normalize(featurize_df(test))
print(X_test)

nr de features este  14
nr de exemple este  1338
[[0.12803688 0.         0.12803688 ... 0.         0.         0.        ]
 [0.07944127 0.         0.         ... 0.         0.         0.05362286]
 [0.07093031 0.         0.         ... 0.03546516 0.         0.00818427]
 ...
 [0.         0.         0.         ... 0.         0.         0.10520384]
 [0.12028661 0.         0.         ... 0.         0.         0.04059673]
 [0.         0.         0.         ... 0.         0.         0.05058793]]


In [1228]:
# from sklearn.model_selection import train_test_split
# X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
# print(X_train2.shape)
# print(X_valid.shape)
# print(np.bincount(y_train2))
# print(np.bincount(y_valid))

In [1229]:
# from sklearn import svm
# from sklearn.metrics import balanced_accuracy_score, accuracy_score

# for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]:
#     print('pentru C = ', C)
#     model = svm.LinearSVC(C=C)
#     model.fit(X_train2, y_train2)
#     # e la fel de bun ca si cum as prezice 0 de fiecare data:
#     preds = model.predict(X_valid)
#     #preds = np.zeros(len(y_valid))
#     print(balanced_accuracy_score(y_valid, preds))
#     print(accuracy_score(y_valid, preds))
  


In [1230]:


# model = KNeighborsClassifier(n_neighbors=7)
# model.fit(X_traincopy, y_traincopy)

# preds = model.predict(X_test)
# print(sum(preds))

In [1231]:
from sklearn.linear_model import Perceptron
#model = svm.LinearSVC(C=1)
model = Perceptron()

#model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_traincopy, y_traincopy)
preds = model.predict(X_test)
#print('accuratete pe train ', balanced_accuracy_score(y_train2, preds))
# print('acuratete pe valid ', balanced_accuracy_score(y_valid, preds))
print(np.sum(preds))


420


# **Creare csv**

In [1232]:
df = pd.DataFrame()
df['id'] = test.index + len(train) + 1
df['complex'] = preds
df.to_csv('submission.csv', index=False)